## Goal:

- Build notebook to do RAG + Tools
- Try further with LangGraph

By: Hasan Rafiq

# Installing the package

In [1]:
!pip install google-cloud-aiplatform langchain langchain-core langchain-google-vertexai faiss-cpu ragas langchainhub --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

# Authenticate again

In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [2]:
PROJECT_ID = "hasanrafiq-test-331814"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [3]:
from google.cloud import aiplatform
# from vertexai.preview.language_models import TextGenerationModel, ChatModel
from vertexai.generative_models import GenerativeModel, Part, ChatSession, Image, FunctionDeclaration, Tool, Content
from langchain_google_vertexai import VertexAI
import http.client
import typing
import urllib.request
import pandas as pd

from google.cloud import aiplatform, storage
from vertexai.preview.language_models import TextGenerationModel, ChatModel, TextEmbeddingModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pprint
import re
import io

from langchain.llms import VertexAI
from langchain import hub
from langchain.text_splitter import TextSplitter, CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader, GCSFileLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain import hub

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness
)
from datasets import Dataset
from ragas import evaluate

aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Implement a RAG

## Source text

In [4]:
document = ["""
Information retrieval using AI and LLMs
Vertex AI Search brings together the power of deep information retrieval, state-of-the-art natural language processing, and the latest in large language model (LLM) processing to understand user intent and return the most relevant results for the user.

With Vertex AI Search, you can build a Google-quality search app on your own data and embed a search bar in your web pages or app.

With Recommendations, you can build a recommendations app on your own data that will suggest content similar to the content that the user is currently viewing.

Note: The generic recommendations feature is a Preview offering, subject to the "Pre-GA Offerings Terms" of the GCP Service Specific Terms. Pre-GA products and features may have limited support, and changes to pre-GA products and features may not be compatible with other pre-GA versions. For more information, see the launch stage descriptions. Further, by using this feature, you agree to the Generative AI Preview terms and conditions ("Preview Terms"). For this feature, you can process personal data as outlined in the Cloud Data Processing Addendum, subject to applicable restrictions and obligations in the Agreement (as defined in the Preview Terms).
An easy experience to get started
Vertex AI Search makes it easy to get started with high-quality search or recommendations based on data that you provide. As part of the setup experience, you can:

Use your existing Google Account or sign up for one.
Use your existing Google Cloud project or create one.
Create an app and attach a data store to it. Provide data to search or recommend by entering the URLs for your website content, importing your data from BigQuery or Cloud Storage, or importing FHIR R4 data from Cloud Healthcare API, or uploading through RESTful CRUD APIs. Syncing data from Jira, Salesforce, or Confluence is available in Preview with allowlist.
Embed JavaScript widgets and API samples to integrate search or recommendations into your website or applications.
Data stores and apps
With Vertex AI Search, you create a search or recommendations app and attach it to a data store. You import your data into a data store and index your data. Apps and data stores have a one-to-one relationship.

There are various kinds of data stores that you can create, based on the type of data you use. Each data store can contain one type of data:

Website data: You can provide domains such as yourexamplewebsite.com/faq and yourexamplewebsite.com/events and enable search or recommendations over the content at those domains.
Structured data: A data store with structured data enables semantic search or recommendations over structured data such as a BigQuery table or NDJSON files. For example, you can enable search or recommendations over a product catalog for your ecommerce experience, a movie catalog for movie search or recommendations, or a directory of doctors for provider search or recommendations.
Unstructured data: An unstructured data store enables semantic search or recommendations over data such as documents and images. For example, a financial institution can enable search or recommendations over their private corpus of financial research publications, or a biotech company can enable search or recommendations over their private repository of medical research.
Healthcare data: A healthcare data store enables semantic search over healthcare FHIR R4 data imported from Cloud Healthcare API. For example, a healthcare provider can search over a patient's clinical history using exploratory queries.
"""]

## Create helper functions

In [5]:
# Initialize Gemini model
model = VertexAI( max_output_tokens=2048, model="gemini-1.0-pro", top_p = 1, temperature = 0)
llm_embeddings = VertexAIEmbeddings()

def chunk_text(input_texts, separator="", chunk_size=100, overlap=20):
  ## Create split of texts on a chunk
  text_splitter = CharacterTextSplitter(separator="", chunk_size=chunk_size, chunk_overlap=overlap)
  chunks = text_splitter.create_documents(input_texts)

  return chunks

def create_vector_store(chunks):
  ## Create vector store from embeddings
  db = FAISS.from_documents(chunks, llm_embeddings)

  return db

# def query_vector_store(query, db, top_n_docs=5):
#   docs = db.similarity_search(query, k=top_n_docs)

#   return docs

def query_vector_store(query, db, top_n_docs, multiquery):
  if multiquery:
    retriever_from_llm = MultiQueryRetriever.from_llm(
      retriever=db.as_retriever(search_kwargs={"k": top_n_docs}), llm=model
    )
    docs = retriever_from_llm.get_relevant_documents(query=query)
  else:
    retriever = db.as_retriever(search_kwargs={"k": top_n_docs})
    docs = retriever.get_relevant_documents(query)

  return docs

def create_with_refine(input_chunks, title):
  prompt_template = """Respond as per following instructions:
  {text}
  CONCISE SUMMARY:"""
  prompt = PromptTemplate.from_template(prompt_template)

  refine_template = (
      """
      You are a QnA bot meant to answer questions ONLY from the contexts provided below and NO PRIOR knowledge

      Question:
      -----------
      {query}
      -----------

      We have provided a response version of the response up to a certain point: {existing_answer}
      You have the opportunity to refine the existing response response with below contexts
      (only if needed) with some more context as specified below.

      Rules:
      ------------
      Given the new context and the query, refine the response further as necessary for answering the query but
      never include the context as is in the response, you should ONLY and ONLY use the knowledge provided in contexts.
      DONT add any information which is not present in contexts.
      Make sure that the content is always relevant to the original query.
      If the context provided isn't useful, dont make any change to the response, just return the current response.
      ------------

      Context:
      ------------
      {text}
      ------------
      """
  ).replace('{query}', title)

  refine_prompt = PromptTemplate.from_template(refine_template)
  chain = load_summarize_chain(
      llm=model,
      chain_type="refine",
      question_prompt=prompt,
      refine_prompt=refine_prompt,
      return_intermediate_steps=True,
      input_key="input_documents",
      output_key="output_text"
  )
  result = chain({"input_documents": input_chunks}, return_only_outputs=True)

  context = []
  for ic in input_chunks:
    context.append( ic.page_content )

  return result['intermediate_steps'], result['output_text'], context

def run_query(query, db, top_n_docs=5, multiquery=False):
  docs = query_vector_store(query, db, top_n_docs, multiquery)

  ## Create article using top N docs
  intermediate_steps, final_text, contexts = create_with_refine(docs, query)

  return intermediate_steps, final_text, contexts

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VertexAI` was deprecated in LangChain 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(
/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `VertexAIEmbeddings` was deprecated in LangChain 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAIEmbeddings`.
  warn_deprecated(


In [6]:
top_n_docs = 6 # @param {type:"integer"} ## Number of similar chunks used to create content

## Create chunks from array of texts
separator = " "
chunks = chunk_text(document, separator, chunk_size=250, overlap=20)
len(chunks)

16

In [7]:
## Create vector store from chunks
db = create_vector_store(chunks)

## Test RAG

In [8]:
model_answers = []
retrieved_contexts = []

In [9]:
query = "What type of data can data stores contain in Vertex AI Search ? Give an elaborated answer" # @param {type:"string"} ## Query

## Create response using top N docs
intermediate_steps, final_text, contexts = run_query(query, db, top_n_docs, multiquery=False)

model_answers.append( final_text )
retrieved_contexts.append( contexts )

final_text

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Vertex AI Search can store various types of data, including:\n\n- **Text:** This includes unstructured text data such as documents, articles, emails, and web pages. Vertex AI Search uses natural language processing (NLP) to understand the meaning of text data and extract relevant information.\n\n\n- **Structured data:** This includes data that is organized in a tabular format, such as spreadsheets, databases, and CSV files. Vertex AI Search can index structured data and make it searchable by specific fields or columns. This enables semantic search or recommendations over structured data such as a BigQuery table or NDJSON files.\n\n\n- **Images:** Vertex AI Search can index and search images based on their visual content. It uses computer vision technology to extract features from images and match them to relevant queries.\n\n\n- **Audio:** Vertex AI Search can index and search audio files, such as podcasts, music, and voice recordings. It uses speech recognition technology to transcr

In [10]:
query = "What does Vertex search support in terms of Healthcare data ?" # @param {type:"string"} ## Query

## Create response using top N docs
intermediate_steps, final_text, contexts = run_query(query, db, top_n_docs, multiquery=False)

model_answers.append( final_text )
retrieved_contexts.append( contexts )

final_text

" **Healthcare Data:**\n- Semantic search over healthcare FHIR R4 data from Cloud Healthcare API.\n- Suitable for healthcare organizations, research institutions, or biotech companies.\n- For example, a healthcare provider can search over a patient's clinical history using exploratory queries.\n- Vertex AI Search can also be used to search over other types of healthcare data, such as medical images, genomic data, and electronic health records.\n- It leverages the power of deep information retrieval, state-of-the-art natural language processing, and the latest in large language model (LLM) processing to understand user intent and retrieve relevant information from healthcare data sources.\n- Vertex AI Search supports structured data, which enables semantic search or recommendations over structured data such as a BigQuery table or NDJSON files.\n- You can import FHIR R4 data from Cloud Healthcare API to enable semantic search over healthcare data."

In [11]:
query = "Which all services are currently in preview in Vertex Search ?" # @param {type:"string"} ## Query

## Create response using top N docs
intermediate_steps, final_text, contexts = run_query(query, db, top_n_docs, multiquery=False)

model_answers.append( final_text )
retrieved_contexts.append( contexts )

final_text

' Vertex AI Search is currently in preview and offers the following services:\n\n- Search: Build a Google-quality search app on your own data and embed a search bar in your web pages or app.\n- Recommendations: Build a recommendations user intent and return the most relevant results for the user.\n- Data sources: Create search indexes from various data sources, including Salesforce, Jira, Confluence, healthcare FHIR R4 data imported from Cloud Healthcare API, and private repositories of medical research.'

# Tools

Tools can be just about anything — APIs, functions, databases, etc. Tools allow us to extend the capabilities of a model beyond just outputting text/messages. The key to using models with tools is correctly prompting a model and parsing its response so that it chooses the right tools and provides the right inputs for them.

https://python.langchain.com/docs/use_cases/tool_use/prompting/

In [12]:
import google.auth

from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from ragas.llms.base import LangchainLLMWrapper
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools.render import render_text_description
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

import os

config = {
    "project_id": "hasanrafiq-test-331814",
}

# authenticate to GCP
creds, _ = google.auth.default(quota_project_id=config['project_id'])

model = ChatVertexAI(top_k=40, top_p=1, temperature=0, credentials=creds)
ragas_vertexai_llm = wrapper = LangchainLLMWrapper(model)
vertexai_embeddings = VertexAIEmbeddings(credentials=creds)

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatVertexAI` was deprecated in LangChain 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import ChatVertexAI`.
  warn_deprecated(


In [13]:
## Create Google search engine - instructions to make your keys are here - https://python.langchain.com/docs/integrations/tools/google_search/

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""

search = GoogleSearchAPIWrapper()

search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

search_tool.run("Obama's first name?")

/root/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


"Full Maiden Name of Mother. STANLEY. 15. Age of Mother 16. 18. Yes. No. 9. Race of Father. OBAMA. African. Usual Occupation. 12b. Student. DUNHAM. Kind of\xa0... Barack Hussein Obama II. (1961-08-04) August 4, 1961 (age 62) Honolulu, Hawaii, U.S. · Democratic · Michelle Robinson. \u200b. ( m. · 1992)\u200b. Apr 2, 2018 ... BARACK : Barkat and Mubarak both are derived from it in Hindi and Urdu. Roughly meaning blessing, abundance etc. Husen or Hussein from which\xa0... At the first-ever Global AI Summit last year, I laid out our vision for a future where AI advances the public interest. To help build that future, I am\xa0... First Lady Michelle LaVaughn Robinson Obama is a lawyer, writer, and the wife of the 44th President, Barack Obama. She is the first African-American First\xa0... Apr 7, 2021 ... No, Obama is secretly his last name, his first name is obviously Joe, though. Joe Obama. Joebama. Jan 19, 2017 ... Hopeful parents named their sons for the first Black president, whose name

In [14]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: float) -> float:
    "Exponentiate the base to the exponent power."
    return base**exponent

@tool
def rag_vertex(question: str) -> str:
    "Answer question related to Vertex AI search and conversation ONLY"

    ## Create response using top N docs
    intermediate_steps, final_text, contexts = run_query(question, db, top_n_docs, multiquery=False)

    return final_text

@tool
def google_search(question: str) -> str:
    "Answer questions related to facts, non-mathematics, but NOT related to Vertex AI search and Conversation"

    ## Create response using top N docs
    res = search_tool.run(question)

    return str(res)

In [15]:
## Tool chooser - Function which helps the model choose in case of multiple functions/

def tool_chooser(model_output):
    tool_map = {tool.name: tool for tool in tools}
    chosen_tool = tool_map[model_output["name"]]
    return itemgetter("arguments") | chosen_tool

In [16]:
## Render the text description of the tools
## These are used internally by the LLM when deciding which tool to chose

tools = [multiply, add, exponentiate, rag_vertex, google_search]
rendered_tools = render_text_description(tools)
rendered_tools

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

print(rendered_tools)

multiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.
add: add(first_int: int, second_int: int) -> int - Add two integers.
exponentiate: exponentiate(base: int, exponent: float) -> float - Exponentiate the base to the exponent power.
rag_vertex: rag_vertex(question: str) -> str - Answer question related to Vertex AI search and conversation ONLY
google_search: google_search(question: str) -> str - Answer questions related to facts, non-mathematics, but NOT related to Vertex AI search and Conversation


In [17]:
## Chain to parse parameters from the function call, returning name of the function to call and its arguments

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}

In [18]:
## Call the function selected above ( using Multiply function specifically )

chain = prompt | model | JsonOutputParser() | itemgetter("arguments") | multiply
chain.invoke({"input": "what's thirteen times 4"})

52

In [19]:
rendered_tools = render_text_description(tools)
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | model | JsonOutputParser() | tool_chooser
chain.invoke("what's 3 plus 1132")

1135

In [20]:
chain.invoke("what's 4.233 power 5 power 3")

1073741824.0

In [21]:
chain.invoke("cube of ninety nine")

970299.0

In [22]:
chain.invoke("cube of ninety nine")

970299.0

In [23]:
chain.invoke("What is the distance between Warsaw and Krakow ?")

"Questions & Answers · Distance: 289 km · Duration: 2h 14m\xa0... You'll usually find 18 trains per day traveling the 250 km between these two destinations. Direct trains are usually available every day on the route to Krakow\xa0... Questions & Answers · Distance: 289 km · Duration: 2h 22m\xa0... Distance between Krakow and Warsaw is 252 kilometers (157 miles) in Poland. Also calculate the driving distance and how far is it the travel time. The distance between Warsaw and Kraków is 180 miles, which takes a minimum of 3 hours 35 minutes. FlixBus has a large nationwide network, so you can travel\xa0... Distance between Warsaw and Krakow is 252 kilometers (157 miles) in Poland. Also calculate the driving distance and how far is it the travel time. Looking how to get from Warsaw to Krakow? Check trip schedule and travel distance. Compare prices for trains, buses, ferries and ✈️ flights. Jun 9, 2023 ... According to Google Maps (Kraków to Warsaw ) it's 283kms. But it's the closest walkable 

In [24]:
chain.invoke("Which all services are currently in preview in Vertex AI Search ?")

' The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search.'

## ReAct Agent for Chat based QnA

- RAG + ReAct based Chatbot
- Uses tools created above for selection

In [25]:
@tool
def rag_vertex(question) -> str:
    "Answer question related to Vertex AI search and conversation ONLY"

    ## Create response using top N docs
    intermediate_steps, final_text, contexts = run_query(question, db, top_n_docs, multiquery=False)

    return final_text

@tool
def google_search(question) -> str:
    "Answer questions related to facts, non-mathematics, but NOT related to Vertex AI search and Conversation"

    ## Create response using top N docs
    res = search_tool.run(question)

    return str(res)

In [26]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt.pretty_print()

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [27]:
prompt_new = """You are a chat bot ! Answer the following questions as best you can.
You have access to the following tools:

{tools}

and your conversation so far( which might have the answer to the question )

Use the following format:

To use a tool, please use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool ? YES
Action: the action to take, should be one of the tools: [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? NO
Final Answer: [your response here]
```

Rules:
1. If tool = 'rag_vertex', then feel free to change the question to only be specific to Vertex AI Search and Conversation
2. Use 'scratch pad' to store intermediate results
3. In every iteration of Thought/Action/Action Input/Observation, refine the 'current response' to improve the response and evaluate if next thought ieration is needed or not ?
4. If the question is completely answered in "current response" then DON'T iterate further

Begin!

Conversation:
{chat_history}

Question: {input}
Thought:{agent_scratchpad}"""

prompt_new = PromptTemplate(
    template=prompt_new,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad", "chat_history"],
)

In [28]:
prompt_new.pretty_print()

You are a chat bot ! Answer the following questions as best you can. 
You have access to the following tools:

{tools}

and your conversation so far( which might have the answer to the question )

Use the following format:

To use a tool, please use the following format:

Question: the input question you must answer
Thought: Do I need to use a tool ? YES
Action: the action to take, should be one of the tools: [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? NO
Final Answer: [your response here]
```

Rules:
1. If tool = 'rag_vertex', then feel free to change the question to only be specific to Vertex AI Search and Conversation
2. Use 

In [63]:
chat_history = ""

In [64]:
llm = VertexAI( max_output_tokens=2048, model="gemini-1.0-pro", top_p = 1, temperature = 0)

tools_react = [rag_vertex, google_search]
# tools_react = [multiply, add, exponentiate, rag_vertex, google_search]

In [65]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools_react, prompt_new)

In [66]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools = tools_react, verbose=True)

resp = agent_executor.invoke({
    "input": "Hi how are you ? How can you help me ?",
    "chat_history": chat_history
})



> Entering new AgentExecutor chain...
 Do I need to use a tool? NO
Final Answer: I am doing great! I am a chatbot assistant designed to help you with questions related to Vertex AI Search and Conversation. I can also provide general information and answer questions on a wide range of topics. How can I help you today?

> Finished chain.


In [67]:
chat_history = f"""
{resp["chat_history"]}
(Human) {resp["input"]}
(Bot) {resp["output"]}
"""

print(chat_history)



(Human) Hi how are you ? How can you help me ?
(Bot) I am doing great! I am a chatbot assistant designed to help you with questions related to Vertex AI Search and Conversation. I can also provide general information and answer questions on a wide range of topics. How can I help you today?



In [68]:
resp = agent_executor.invoke({
    "input": "Which all services are currently in preview in Vertex AI Search ? Secondly what is the age of Earth ?",
    "chat_history": chat_history
})



> Entering new AgentExecutor chain...
 Do I need to use a tool? YES
Action: rag_vertex
Action Input: Which all services are currently in preview in Vertex AI Search ? The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search. Do I need to use a tool? YES
Action: google_search
Action Input: what is the age of Earth ?Earth is estimated to be 4.54 billion years old, plus or minus about 50 million years. Scientists have scoured the Earth searching for the oldest rocks to ... Age of Earth · The age of Earth is estimated to be 4.54 ± 0.05 billion years (4.54 × 109 years ± 1%). · Following the development of radiometric age-dating in ... Oct 19, 2023 ... The examination and analysis of rocks on Earth's surface, and of extraterrestrial rocks, have enabled scientists to determine the ... Jul 9, 2007 ... The results show that the meteorites, and therefore the Solar System, formed between 4.53 and 4.58 billion years ago. ... These calculation

In [69]:
chat_history = f"""
{resp["chat_history"]}
(Human) {resp["input"]}
(Bot) {resp["output"]}
"""

print(chat_history)




(Human) Hi how are you ? How can you help me ?
(Bot) I am doing great! I am a chatbot assistant designed to help you with questions related to Vertex AI Search and Conversation. I can also provide general information and answer questions on a wide range of topics. How can I help you today?

(Human) Which all services are currently in preview in Vertex AI Search ? Secondly what is the age of Earth ?
(Bot) The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search. Additionally, Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.



In [70]:
resp = agent_executor.invoke({
    "input": "What is the age of Earth ?",
    "chat_history": chat_history
})



> Entering new AgentExecutor chain...
 Do I need to use a tool? NO
Final Answer: Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.

> Finished chain.


In [71]:
chat_history = f"""
{resp["chat_history"]}
(Human) {resp["input"]}
(Bot) {resp["output"]}
"""

print(chat_history)





(Human) Hi how are you ? How can you help me ?
(Bot) I am doing great! I am a chatbot assistant designed to help you with questions related to Vertex AI Search and Conversation. I can also provide general information and answer questions on a wide range of topics. How can I help you today?

(Human) Which all services are currently in preview in Vertex AI Search ? Secondly what is the age of Earth ?
(Bot) The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search. Additionally, Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.

(Human) What is the age of Earth ?
(Bot) Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.



In [72]:
resp = agent_executor.invoke({
    "input": "Which all services are currently in preview in Vertex AI Search ?",
    "chat_history": chat_history
})



> Entering new AgentExecutor chain...
 Do I need to use a tool? YES
Action: rag_vertex
Action Input: Which all services are currently in preview in Vertex AI Search ? The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search. I now know the final answer
Final Answer: The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search.

> Finished chain.


In [73]:
chat_history = f"""
{resp["chat_history"]}
(Human) {resp["input"]}
(Bot) {resp["output"]}
"""

print(chat_history)






(Human) Hi how are you ? How can you help me ?
(Bot) I am doing great! I am a chatbot assistant designed to help you with questions related to Vertex AI Search and Conversation. I can also provide general information and answer questions on a wide range of topics. How can I help you today?

(Human) Which all services are currently in preview in Vertex AI Search ? Secondly what is the age of Earth ?
(Bot) The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search. Additionally, Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.

(Human) What is the age of Earth ?
(Bot) Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.

(Human) Which all services are currently in preview in Vertex AI Search ?
(Bot) The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search.



In [74]:
resp = agent_executor.invoke({
    "input": "What is the distance between Krakow and Vienna ?",
    "chat_history": chat_history
})



> Entering new AgentExecutor chain...
 Do I need to use a tool ? YES
Action: google_search
Action Input: distance between Krakow and ViennaThe cheapest way to get from Kraków to Vienna costs only €17, and the quickest way takes just 2½ hours. Find the travel option that best suits you. The average journey time by train between Krakow Central and Vienna is 6 hours and 26 minutes, with around 7 trains per day. The journey time may be longer on ... The cheapest way to get from Vienna to Kraków costs only 83 zł, and the quickest way takes just 3 hours. Find the travel option that best suits you. Distance between Krakow and Vienna is 331 kilometers (206 miles). Driving distance from Krakow to Vienna is 464 kilometers (288 miles). Jan 21, 2023 ... This route will take you through the Czech Republic, at a distance of around 465 kilometers, the Krakow to Vienna distance takes around 5–6 ... Getting from Kraków to Vienna is cheap and easy when you travel with FlixBus. The two cities are 291 m

In [75]:
chat_history = f"""
{resp["chat_history"]}
(Human) {resp["input"]}
(Bot) {resp["output"]}
"""

print(chat_history)







(Human) Hi how are you ? How can you help me ?
(Bot) I am doing great! I am a chatbot assistant designed to help you with questions related to Vertex AI Search and Conversation. I can also provide general information and answer questions on a wide range of topics. How can I help you today?

(Human) Which all services are currently in preview in Vertex AI Search ? Secondly what is the age of Earth ?
(Bot) The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search. Additionally, Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.

(Human) What is the age of Earth ?
(Bot) Earth is estimated to be 4.54 billion years old, with a margin of error of approximately 50 million years.

(Human) Which all services are currently in preview in Vertex AI Search ?
(Bot) The generic recommendations feature and the healthcare data store are currently in preview in Vertex AI Search.

(Human) Wh